# JEM092 Asset Pricing
# Seminar 7
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 12.4.2022

This seminar is recycled from the 2019/2020 course Portfolio Analysis and Risk Management. Authors of the original seminars are **Martin Hronec** and **Marek Hauzr**.

In [ ]:
# Import packages.
library(quantmod)

# Multifactor models

Graham and Harvey (2001): "While the CAPM is popular, we show later that it is not clear that the model is applied properly in practice. Of course, even if it is applied properly, it is not clear that the CAPM is a very good model."

Thus APT and multifactor models.

Empirical factors used in factor models fall into the following three categories:
1. External (macroeconomic) factors, such as gross domestic product (GDP), consumer price index (CPI), unemployment rate, credit spreads on bonds, and the steepness of the yield curve.
2. Fundamental factors (firm characteristics), such as the price-earnings ratio, the dividend-payout ratio, the earnings growth forecast, and financial leverage.
3. Extracted (statistical) factors, such as the return on the market portfolio (computed as the compilation of returns on the individual securities), the average of the returns of stocks in a particular industry (utilities, transportation, aerospace), and so on. Some such factors are computed using principal component analysis and similar methods. 

Currently, the FF5 model is considered a “standard” in asset pricing literature. However, a lot of problems arise with it and thus other models are coming up currently.

SMB is the return spread of small minus large stocks (i.e. the size effect).
HML is the return spread of cheap minus expensive stocks (i.e. the value effect).
RMW is the return spread of the most profitable firms minus the least profitable.
CMA is the return spread of firms that invest conservatively minus aggressively.

In [ ]:
# Clear your workspace.
rm(list = ls()) ; par(mfrow = c(1,1))

# Download data.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Portfolios_Formed_on_ME_TXT.zip",
    destfile = "Portfolios_Formed_on_ME.zip"
)
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Portfolios_Formed_on_INV_TXT.zip",
    destfile = "Portfolios_Formed_on_INV.zip"
)
unzip("F-F_Research_Data_Factors.zip")
unzip('Portfolios_Formed_on_ME.zip')
unzip('Portfolios_Formed_on_INV.zip')

# Load factors.
FF3factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1067,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
head(FF3factors)

In [ ]:
portfolio <- read.delim(
    'Portfolios_Formed_on_ME.txt',
    col.names = c(
        "t",
        "smaller.0",
        "Lo.30",
        "Med.40",
        "Hi.30",
        "Lo.20",
        "Qnt.2",
        "Qnt.3",
        "Qnt.4",
        "Hi.20",
        "Lo.10",
        "Dec.2",
        "Dec.3",
        "Dec.4",
        "Dec.5",
        "Dec.6",
        "Dec.7",
        "Dec.8",
        "Dec.9",
        "Hi.10"
    ),
    sep = '',
    nrows = 1067,
    header = FALSE,
    skip = 13,
    stringsAsFactors = FALSE
)
head(portfolio)

In [ ]:
# Construct the excess returns for the portfolio.
portfolio.rf <- portfolio - FF3factors$rf

# Combine the samples.
sample <- cbind(portfolio.rf, FF3factors)

# Generate timestamps.
dates <- seq(as.Date('1926-07-01'), as.Date('2015-05-01'), by = 'month')

# Combine timestamps with the sample.
sample <- cbind(dates, sample)

lm.Dec.2 <- lm('Dec.2 ~ mkt.rf + smb + hml', data = sample)
summary(lm.Dec.2)

In [ ]:
# Set the number of portfolios.
n <- 20

# Create the table of coefficients.
coeffs <- as.data.frame(matrix(nrow = 2 * 4 + 2, ncol = n))

# Specify the names of the columns of the table of coefficients.
names(coeffs) <- names(sample)[2:(n + 1)]
attributes(coeffs)$row.names <- c(
    "Intercept",
    "t.Intercept",
    "mkt.rf",
    "t.mkt.rf",
    "smb",
    "t.smb",
    "hml",
    "t.hml",
    "Obs",
    "R-squared"
)

# Estimate FF3 model.
for (i in 1:n) {
    # Estimate the model.
    lm <- lm(sample[,(i + 1)] ~ mkt.rf + smb + hml, data = sample)
    # Paste the coefficients into the right rows.
    coeffs[c(1, 3, 5, 7), i] <- summary(lm)$coefficients[, 1]
    # Paste the t-statistics into the right rows.
    coeffs[c(2, 4, 6, 8), i] <- summary(lm)$coefficients[, 3]
    # Paste the number of used observations into the right row.
    coeffs[9, i] <- length(lm$model[, 1])
    # Paste the R-squared into the right row.
    coeffs[10, i] <- summary(lm)$r.squared
}
# Round the results for better readability.
coeffs <- round(coeffs, digits = 3)

# See the coefficients at the quintiles.
coeffs[, 6:10]

In [ ]:
# See the coefficients at the deciles.
coeffs[, 11:20]

In [ ]:
# Let's try it for some specific stock.
index <- '^GSPC'
stock <- 'BRK-A'
time <- '2009-01-01'

getSymbols(index, from = time)
getSymbols(stock, from = time)

index.data <- dailyReturn(GSPC$GSPC.Adjusted, type = 'log')
stock.data <- dailyReturn(get(stock), type = 'log')

data <- merge(stock.data, index.data)
lm.beta <- summary(lm(data[, 1] ~ data[, 2]))

summary(lm(data[, 1] ~ data[, 2]))

# References

Graham, J. R., & Harvey, C. R. (2001). The theory and practice of corporate finance: Evidence from the field. Journal of financial economics, 60(2-3), 187-243.